## Analysis of Lewd Conduct Calls

by Stacy Montemayor

An examination of how longer time deltas between when the complaint was received and when an officer arrived on scene affects outcomes of lewd conduct calls.

In [1]:
import pandas as pd
import numpy as np

from ipysankeywidget import SankeyWidget

In [2]:
%matplotlib inline

### Import cleaned data

In [3]:
%run import_clean_call_data.ipynb

### Analysis

In [4]:
links = [
    {'source': 'start', 'target': 'A', 'value': 2},
    {'source': 'A', 'target': 'B', 'value': 2},
    {'source': 'C', 'target': 'A', 'value': 2},
    {'source': 'A', 'target': 'C', 'value': 2},
]

In [5]:
w = SankeyWidget(links=links, margins=dict(top=0, bottom=0, left=50, right=100))

Cleaning requirements: 
- remove all but lewd conduct calls
- figure out how to determine if they're happening currently-ish
- Create ~5 time buckets
- Flow each precinct into a time bucket
- Then flow into how event was cleared


What I want to see: 
scatterplot of timedelta by precinct  
time deltas by precinct

In [12]:
# Create dataframe that will contain all lewd conduct calls 
lewd_conduct_df = pd.DataFrame(data)

# Create dataframe containing only lewd conduct calls
lewd_conduct_df = lewd_conduct_df[is_lewd_conduct]